In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycytominer import aggregate, annotate, normalize, feature_select, cyto_utils
from cytominer_eval import evaluate
from cytominer_eval.transform import metric_melt
from pycytominer.cyto_utils import infer_cp_features
import seaborn as sns
from sklearn.decomposition import PCA
import sys
sys.path.insert(0, "/Users/mbornhol/git/neural-profiling")
from utils import eval

In [2]:
index = pd.read_csv("../00_index.csv")

# Unknown level 3. Prob raw CP data
Whatever is in
data_loc = '/Users/mbornhol/git/lincs-cell-painting/profiles/2016_04_01_a549_48hr_batch1/'

In [3]:
full = pd.read_csv('/Users/mbornhol/git/neural-profiling/baseline/01_data/level_3_data/full_level3.csv')
df_sub = pd.merge(full, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
unk = full[df_sub._merge == "both"]

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
res = evaluate(
    profiles=unk.sample(frac=0.2),
    features=infer_cp_features(unk),
    meta_features=infer_cp_features(unk, metadata=True),
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

In [10]:
# ONly DMSOs
unk_dmso = unk[unk["Metadata_broad_sample"] == "DMSO"]

In [6]:
res = evaluate(
    profiles=_.sample(frac=0.5),
    features=infer_cp_features(unk),
    meta_features=infer_cp_features(unk, metadata=True),
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999912,1.389905,6.224012e-17


In [11]:
res = evaluate(
    profiles=unk_dmso.sample(frac=0.5),
    features=infer_cp_features(unk),
    meta_features=infer_cp_features(unk, metadata=True),
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999911,5.578862,2.139227e-190


In [12]:
res = evaluate(
    profiles=unk_dmso.sample(frac=0.5),
    features=infer_cp_features(unk),
    meta_features=infer_cp_features(unk, metadata=True),
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res


,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999912,3.445542,0.0


# pretrained

In [13]:
eff = pd.read_csv('/Users/mbornhol/git/neural-profiling/00_thesis/pre_trained/aggregated_efficientnet_median.csv', low_memory=False)

In [14]:
merge = pd.merge(eff, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df = eff[merge._merge == "both"]
df.shape

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(8733, 6418)

In [15]:
features = [f for f in df.columns if f.startswith("eff")]

In [46]:
sphere = normalize(
    profiles=df,
    features=features,
    samples="Metadata_broad_sample == 'DMSO'",
    method='spherize',
    output_file='none'
)

In [66]:
df = df.sample(frac=0.2)
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.996716,3.071389,1.140804e-111


In [53]:
df = sphere.sample(frac=1)
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.329309,4.061539,6.270812e-54


In [16]:
# ONly DMSOs
pre_dmso = df[df["Metadata_broad_sample"] == "DMSO"]

In [20]:
res = evaluate(
    profiles=pre_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.997541,2.102206,3.691406e-99


In [21]:
res = evaluate(
    profiles=pre_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.99752,58.094764,0.0


In [22]:
res = evaluate(
    profiles=pre_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

AssertionError: replicate_group not found in melted dataframe columns

In [24]:
pre_dmso.Metadata_Batch_Number

AttributeError: 'DataFrame' object has no attribute 'Metadata_Batch_Number'

#   trained

In [25]:
trained = pd.read_csv('/Users/mbornhol/git/neural-profiling/training/runs/1028/1028_aggregated_median.csv', low_memory=False)

In [26]:
merge = pd.merge(trained, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df_trained = trained[merge._merge == "both"]
df_trained.shape

(8745, 1298)

In [27]:
features = [f for f in df_trained.columns if f.startswith("eff")]

In [57]:
sphere = normalize(
    profiles=df,
    features=features,
    samples="Metadata_broad_sample == 'DMSO'",
    method='spherize',
    output_file='none'
)

In [59]:
df = df.sample(frac=0.1)
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.531583,4.043584,1.562464e-51


In [74]:
# df = df.sample(frac=0.1)
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.525372,84.117144,0.0


In [60]:
df = sphere.sample(frac=0.1)
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.223091,4.914371,4.785025e-75


In [28]:
train_dmso = df_trained[df_trained["Metadata_broad_sample"] == "DMSO"]

In [29]:
res = evaluate(
    profiles=train_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.675098,1.677526,1.539618e-43


In [31]:
res = evaluate(
    profiles=train_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.672271,187.879943,0.0


In [ ]:
res = evaluate(
    profiles=pre_dmso.sample(frac=0.5),
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

In [67]:
# Testing Plate corr.
res = evaluate(
    profiles=unk.sample(frac=0.2),
    features=infer_cp_features(unk),
    meta_features=infer_cp_features(unk, metadata=True),
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.99989,3.792384,2.454376e-106


In [69]:
res = evaluate(
    profiles=df,
    features=features,
    meta_features=[f for f in df.columns if not f.startswith("eff")],
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=[0.995],
)
res


,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.996716,31.605624,0.0


# CP spherized

In [ ]:
df = pd.read_csv('/Users/mbornhol/git/neural-profiling/00_thesis/baseline/2016_04_01_a549_48hr_batch1_dmso_spherized_profiles_with_input_normalized_by_dmso_consensus_median.csv.gz')
df= df.rename(columns= {"Metadata_pert_well" : "Metadata_Well"})

In [ ]:
merge = pd.merge(df, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df = df[merge._merge == "both"]
# I am stuck cause this doesnt have plates

In [ ]:
# res = evaluate(
#     profiles=df.sample(frac=0.1),
#     features=infer_cp_features(df),
#     meta_features=infer_cp_features(df, metadata=True),
#     replicate_groups=['Metadata_pert_well'],
#     operation="enrichment",
#     similarity_metric="pearson",
#     enrichment_percentile=[0.995, 0.99, 0.98],
# )
# res